In [4]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Atiksha10%',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/JASIKA/Documents/Data Analysis/Projects/E-Commerce_data_analysis/Dataset'

# Function to map pandas dtypes to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

try:
    for csv_file, table_name in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        
        print(f"\n🔄 Processing: {csv_file}")
        df = pd.read_csv(file_path)

        # Replace NaN with None to handle SQL NULLs
        df = df.where(pd.notnull(df), None)

        # Clean column names
        df.columns = [col.strip().replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Create table dynamically
        columns_def = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_def})'
        cursor.execute(create_query)

        # Insert data
        insert_query = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(df.columns))})"

        # Batch insert to reduce overhead
        data = [tuple(None if pd.isna(x) else x for x in row) for row in df.to_numpy()]
        cursor.executemany(insert_query, data)

        conn.commit()
        print(f"✅ Inserted {len(data)} records into `{table_name}`.")

except mysql.connector.Error as err:
    print(f"❌ MySQL Error: {err}")
    conn.rollback()

except Exception as e:
    print(f"❌ Unexpected Error: {e}")
    conn.rollback()

finally:
    cursor.close()
    conn.close()
    print("\n🔚 Connection closed.")



🔄 Processing: customers.csv
✅ Inserted 99441 records into `customers`.

🔄 Processing: orders.csv
✅ Inserted 99441 records into `orders`.

🔄 Processing: sellers.csv
✅ Inserted 3095 records into `sellers`.

🔄 Processing: products.csv
✅ Inserted 32951 records into `products`.

🔄 Processing: geolocation.csv
✅ Inserted 1000163 records into `geolocation`.

🔄 Processing: payments.csv
✅ Inserted 103886 records into `payments`.

🔄 Processing: order_items.csv
✅ Inserted 112650 records into `order_items`.

🔚 Connection closed.
